In [5]:
# https://www.opcito.com/blogs/extracting-text-from-images-with-tesseract-ocr-opencv-and-python
# https://www.geeksforgeeks.org/python-reading-contents-of-pdf-using-ocr-optical-character-recognition/
# https://felipemeganha.medium.com/extra%C3%A7%C3%A3o-de-informa%C3%A7%C3%B5es-do-pdf-com-python-opencv-e-pytesseract-53bf8cbbd364
# https://nanonets.com/blog/extract-data-from-pdf/
# http://br-linux.org/linux/tesseract-ocr-agora-suporta-oficialmente-o-portugues-brasileiro


#!pip install opencv-python
#!pip install pytesseract
#!pip install pdf2image
#!pip install Pillow
#sudo apt-get install poppler-utils

[sudo] password for info: 


In [ ]:
# https://www.geeksforgeeks.org/python-reading-contents-of-pdf-using-ocr-optical-character-recognition/
# Requires Python 3.6 or higher due to f-strings

In [25]:
# Import libraries
import platform
from tempfile import TemporaryDirectory
from pathlib import Path
 
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import cv2

In [10]:
if platform.system() == "Windows":
    # We may need to do some additional downloading and setup...
    # Windows needs a PyTesseract Download
    # https://github.com/UB-Mannheim/tesseract/wiki/Downloading-Tesseract-OCR-Engine
 
    pytesseract.pytesseract.tesseract_cmd = (
        r"C:\Program Files\Tesseract-OCR\tesseract.exe"
    )
 
    # Windows also needs poppler_exe
    path_to_poppler_exe = Path(r"C:\.....")
     
    # Put our output files in a sane place...
    out_directory = Path(r"~\Desktop").expanduser()
else:
    out_directory = Path("~").expanduser() 

In [12]:
out_directory = "/home/info/MyNotebooks/Datasets/SentencasTRT1/IMGs"
out_directory

'/home/info/MyNotebooks/Datasets/SentencasTRT1/IMGs'

In [19]:
file_path = "/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/"
#file_name = "01007085520215010071-DEJT-30-05-2022.pdf"
file_name = "0065500-14.2007.5.01.0002.pdf"

In [39]:
# Path of the Input pdf
#PDF_file = Path(r"d.pdf")
PDF_file = Path(file_path+file_name)
print(PDF_file)
 
# Store all the pages of the PDF in a variable
image_file_list = []
 
text_file = out_directory / Path("out_text.txt")
 
def main():
    ''' Main execution point of the program'''
    with TemporaryDirectory() as tempdir:
        # Create a temporary directory to hold our temporary images.
 
        """
        Part #1 : Converting PDF to images
        """
 
        if platform.system() == "Windows":
            pdf_pages = convert_from_path(PDF_file, 500, poppler_path=path_to_poppler_exe)
        else:
            pdf_pages = convert_from_path(PDF_file, 500)
        # Read in the PDF file at 500 DPI
 
        # Iterate through all the pages stored above
        for page_enumeration, page in enumerate(pdf_pages, start=1):
            # enumerate() "counts" the pages for us.
 
            # Create a file name to store the image
            filename = f"{tempdir}\page_{page_enumeration:03}.jpg"
 
            # Declaring filename for each page of PDF as JPG
            # For each page, filename will be:
            # PDF page 1 -> page_001.jpg
            # PDF page 2 -> page_002.jpg
            # PDF page 3 -> page_003.jpg
            # ....
            # PDF page n -> page_00n.jpg
 
            # Save the image of the page in system
            page.save(filename, "JPEG")
            image_file_list.append(filename)
 

        """
        Part #2 - Recognizing text from the images using OCR
        """
        with open(text_file, "a") as output_file:
            # Open the file in append mode so that
            # All contents of all images are added to the same file
 
            # Iterate from 1 to total number of pages
            for image_file in image_file_list:
 
                # Set filename to recognize text from
                # Again, these files will be:
                # page_1.jpg
                # page_2.jpg
                # ....
                # page_n.jpg
 
                # Recognize the text as string in image using pytesserct
                #text = str(((pytesseract.image_to_string(Image.open(image_file)))))
        
                # NEW:
        
                image = cv2.imread(image_file)
                #converting image into gray scale image
                gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

                # converting it to binary image by Thresholding
                # this step is require if you have colored image because if you skip this part
                # then tesseract won't able to detect text correctly and this will give incorrect result
                threshold_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
                custom_config = r'--oem 3 --psm 6'

                details = pytesseract.image_to_data(threshold_img, config=custom_config, lang='por') # output_type=Output.DICT,
                
                parse_text = []
                word_list = []
                last_word = ''

                #for word in details['text']:
                for i, word in enumerate(details):
                    if word != '':
                        word_list.append(word)
                        last_word = word
                    if (last_word != '' and word == '') or (word == details[-1]):
                        parse_text.append(word_list)
                        word_list = []
    
                text = ' '.join(parse_text)
                #text = ''.join(''.join(l) for l in parse_text)
                #text = ' '.join(l for l in parse_text)
                
                # END NEW
                
                # The recognized text is stored in variable text
                # Any string processing may be applied on text
                # Here, basic formatting has been done:
                # In many PDFs, at line ending, if a word can't
                # be written fully, a 'hyphen' is added.
                # The rest of the word is written in the next line
                # Eg: This is a sample text this word here GeeksF-
                # orGeeks is half on first line, remaining on next.
                # To remove this, we replace every '-\n' to ''.
                text = text.replace("-\n", "")
 
                # Finally, write the processed text to the file.
                output_file.write(text)
 
            # At the end of the with .. output_file block
            # the file is closed after writing all the text.
        # At the end of the with .. tempdir block, the
        # TemporaryDirectory() we're using gets removed!       
    # End of main function!
     

if __name__ == "__main__":
      # We only want to run this if it's directly executed!
    main()



/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/0065500-14.2007.5.01.0002.pdf


TypeError: sequence item 0: expected str instance, list found

In [32]:
main()

In [22]:
!echo $TESSDATA_PREFIX